# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [23]:
import nest_asyncio
nest_asyncio.apply()

In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [26]:
from utils.models_definer import ModelName

# storing or not
OCR_TRESHOLD = 0.98
TOP_K = 3
NUM_BATCH_EVAL = 10
STORE = False
DELETE = False
EVAL = False
model_name = ModelName.AZURE_OPENAI
model_name_eval = ModelName.OPENAI

## **LLM Models Define**

In [27]:
from llama_index.core import Settings
from utils.models_definer import get_llm_and_embedding

# ollama/openai/azure_openai
llm,embedding_llm = get_llm_and_embedding(model_name=model_name)

llm_eval = get_llm_and_embedding(model_name=model_name_eval)[0]


Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [28]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes

path = './data'

if STORE:
    docs = read_documents(path=path, ocr_treshold=OCR_TRESHOLD)
    nodes = parse_nodes(documents=docs, llm=llm)

## **Indexing & Storing**

In [29]:
from utils.index_store import (store_vector_index, load_vector_index)

# store
if STORE:
    vector_index = store_vector_index(nodes=nodes,embed_model=embedding_llm, delete=DELETE)
# load
else:
    vector_index = load_vector_index()

Loading the vector index completed.


## **Querying**

### **Retriever**

In [30]:
vector_retriever = vector_index.as_retriever(similarity_top_k=TOP_K)

### **Prompt**

In [31]:
from llama_index.core import PromptTemplate

qa_prompt = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, \
answer the query asking about banking compliance in Indonesia. 
Answer the question based on the context information.
ALWAYS ANSWER WITH USER'S LANGUAGE.
Please provide your answer with [regulation_number](file_url) in metadata 
(if possible) in the following format:

---------------------
Answer... \n\n
Source: [regulation_number](file_url) \n
---------------------

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl = PromptTemplate(qa_prompt)

### **Query Engine**

In [32]:
from llama_index.core.query_engine import RetrieverQueryEngine

vector_query_engine = RetrieverQueryEngine.from_args(retriever=vector_retriever,llm=llm,)

In [33]:
vector_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [34]:
from utils.prompt_display import display_prompt_dict

prompts_dict = vector_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query asking about banking compliance in Indonesia. 
Answer the question based on the context information.
ALWAYS ANSWER WITH USER'S LANGUAGE.
Please provide your answer with [regulation_number](file_url) in metadata 
(if possible) in the following format:

---------------------
Answer... 


Source: [regulation_number](file_url) 

---------------------

Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [41]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # Peraturan Pemerintah Nomor 12 Tahun 2004
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ dari sedan?" # Rp. 140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.
q = "Kapan, dimana, dan oleh siapa surat nomor PER- 06/BL/2012 ditetapkan?" # Surat nomor PER-06/BL/2012 ditetapkan pada tanggal 22 November 2012 di Jakarta oleh Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan.
r = "Apa kepanjangan PSAK?" # Pernyataan Standar Akuntansi Keuangan
s = "Apa itu 'shahibul maal'?" # Pemilik dana pihak ketiga

query_str = k

In [42]:
from llama_index.core.response.notebook_utils import display_response

response_vector = vector_query_engine.query(query_str)
display_response(response_vector)

**`Final Response:`** Nilai SWDKLLJ untuk sedan adalah Rp140.000,00 (seratus empat puluh ribu rupiah).

Source: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf)

### **Chat Engine**

In [37]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [38]:
# from llama_index.agent.openai import OpenAIAgent
# from llama_index.core.agent.legacy.react.base import ReActAgent
from llama_index.core.chat_engine import CondenseQuestionChatEngine

chat_engine = CondenseQuestionChatEngine.from_defaults(
    llm=llm,
    memory=memory,
    query_engine=vector_query_engine,
    verbose=True,
)

In [39]:
from llama_index.core.response.notebook_utils import display_response

response = chat_engine.chat(message=query_str)
display_response(response)

Querying with: Apa itu LJKNB?


**`Final Response:`** LJKNB stands for Lembaga Jasa Keuangan Nonbank, which refers to nonbank financial institutions in Indonesia. These institutions are required to implement risk management in the use of information technology, and they must establish an Information Technology Steering Committee if their total assets exceed Rp1,000,000,000,000,00 (one trillion rupiah). The committee should consist of directors responsible for IT units, directors or officials in charge of risk management, the highest-ranking officials in charge of IT units, and the highest-ranking officials in charge of IT user units.

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for node in response.source_nodes:
    display_source_node(node, show_source_metadata=True)    

**Node ID:** e840f3a2-cc99-4719-b933-b0b79b5ec7ad<br>**Similarity:** 0.8018614649772644<br>**Text:** MENTERI KEUANGAN
REPUBLIK INDONESIA
-4-
 Mobil penumpang angkutan umum sampai dengan 1600 cc
g. B...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'retrieval_score': 0.9084663987159729}<br>

**Node ID:** 10989a33-e3d7-4d3d-8f5c-88ad29732ba8<br>**Similarity:** 0.707891047000885<br>**Text:** MENTERI KEUANGAN
REPUBLIK INDONESIA
memperoleh santunan sebesar Rp 25.000.000,00 (dua puluh lima
...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'retrieval_score': 0.9090815782546997}<br>

**Node ID:** 67d00445-e031-4d07-a3ed-a224c28a62a3<br>**Similarity:** 0.6666857600212097<br>**Text:** REPUBLIK INDONESIA
 Penyelenggaraan Usaha Perasuransian (Lembaran Negara Republik
Indonesia Tahun...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'retrieval_score': 0.8981241583824158}<br>

In [ ]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)

## **Evaluation**

In [81]:
import os

# check directory exist
if not os.path.exists("./json_data"):
    os.makedirs("./json_data")

In [82]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
if EVAL:
    if STORE:
        qa_dataset = generate_question_context_pairs(
            nodes=nodes,
            llm=llm_eval,
            num_questions_per_chunk=2,
        )
        qa_dataset.save_json("./json_data/pg_eval_dataset.json")
    else:
        qa_dataset = EmbeddingQAFinetuneDataset.from_json("./json_data/pg_eval_dataset.json")

### **Retrieval Evaluation**

In [83]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

In [84]:
from utils.evaluator import get_retrieval_eval_df

if EVAL:
    vector_retrieval_eval_results = await get_retrieval_eval_df(name=f"Top-{TOP_K} Eval", metrics=metrics, retriever=vector_retriever, qa_dataset=qa_dataset)
    display(vector_retrieval_eval_results)

### **Response Evaluation**

In [85]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
)

if EVAL:
    # model for evaluation
    relevancy_evaluator = RelevancyEvaluator(llm=llm_eval)
    faithfullness_evaluator = FaithfulnessEvaluator(llm=llm_eval)

#### **Faithfulness Evaluator**

Evaluate Response

In [86]:
from utils.evaluator import get_response_eval_df

if EVAL:
    vector_faithfullness_eval_result = faithfullness_evaluator.evaluate_response(response=response_vector, query=query_str)
    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=vector_faithfullness_eval_result))

Evaluate Source Nodes

In [87]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=faithfullness_evaluator)
    display(relevancy_eval_sources)

#### **Relevancy Evaluator**

Evaluate Response

In [88]:
if EVAL:
    relevancy_eval_result = relevancy_evaluator.evaluate_response(
        query=query_str, response=response_vector
    )

    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=relevancy_eval_result))

Evaluate Source Nodes

In [89]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=relevancy_evaluator)
    display(relevancy_eval_sources)

#### **Batch Evaluator (Faithfulness, Relevancy, Correctness)**

In [90]:
from llama_index.core.evaluation import BatchEvalRunner
from utils.evaluator import get_batch_eval_results
from utils.evaluator import get_batch_eval_df

if EVAL:
    runner = BatchEvalRunner(
        {
            "faithfulness": faithfullness_evaluator, 
            "relevancy": relevancy_evaluator,
        },
        workers=8,
    )

    vector_eval_results = await get_batch_eval_results(runner=runner, qa_dataset=qa_dataset, query_engine=vector_query_engine, num_queries=NUM_BATCH_EVAL)
    display(get_batch_eval_df(vector_eval_results))